In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/Legend13")
import torch
import torch.nn as nn
!pip install torchinfo
from torchinfo import summary

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!jupyter nbconvert --to markdown /content/drive/MyDrive/Legend13/Self_Implement/psm_MobileNet_V1-3.ipynb

[NbConvertApp] Converting notebook /content/drive/MyDrive/Legend13/Self_Implement/psm_MobileNet_V1-3.ipynb to markdown
[NbConvertApp] Writing 28937 bytes to /content/drive/MyDrive/Legend13/Self_Implement/psm_MobileNet_V1-3.md


## **MobileNet V1**

<br/>

- For resource constrained environment such as mobile devices, it is very important to build computationally efficient networks.
- MobileNet v1 efficiently trade-off between efficiency and accuracy by adopting depthwise separable convolutional filters.

<br/>

### **Depthwise Separable Convolution**

<br/>

&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/421bbbfb-1e45-42f0-8993-b4ac0833b506" width="410">

<br/>

1. **Depthwise Convolution**
    - Idential to the grouped convolution with cardinality equal to total channel size.
    - Only performs spatial convolution where each channel of the input requires a single filter and concatenates the resultant feature maps.
    - Each feature map contains spatial representation within one chanenl.
    - This reduces the network complexities, as each filter is only responsible for convolving with its corresponding input channel.


2. **Separable Convolution**
    - Use 1x1 convolution filter to perform channel-wise convolution and linearly combines each input channel into different sets of feature maps.

- Two of convolutions combined can perform roughly identical operation as traditional convolution with no significant decrease in accuracy, but dramatically reduces the computational cost.

<br/>


### **Comparison of Computational Cost**

<br/>

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/8157ce73-23a0-4378-a942-cc3ae2dc534d" width="380">

<br/>

- H, W : input shape per channel
- C : the number of input channels
- F : the number of output features
- K : kernel size of each filter
- **standard convolution**
    - H x W x K x K x C x F
- **depthwise separable convolution**
    - H x W x K x K x C + H x W x C x F

- H x W x K x K x C + H x W x C x F / (H x W x K x K x C x F)
    - 1/F + 1/K^2

<br/>

### **Trade-Off : Accuracy vs Complexity**

<br/>

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/18429f53-cc02-49d9-990d-246ddd65e8f6" width="400">

<br/>

1. **Width Multipliers**
    - $\alpha \in (0, 1]\$
    - simple hyperparameter $\large \alpha$ to regulate the width of the networks
    - multiplied to the number of channel at each layer

2. **Resolution Multipliers**
    - $\rho \in (0, 1]\$
    - adjust the resolution (size of input)
    - affects the representation
    - no change in the number of parameters

- Can make faster but weaker model by simply applying a few hyperparameters. Setting $\large \alpha$ and $\large \rho$ as 1 equals to a baseline.

<br/>

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/3c1f9959-67e2-4cee-9295-f4fc60ac689c" width="400">

<br/>

### **MobileNet Architecture**

<br/>

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/c3ebef6a-4edf-405b-82dc-499747b54e1c"  width="480">


<br/>

### **MobileNet V1 Comparison to Other Models**

<br/>

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/739ef37d-0c41-4f5c-b63b-8b35d7a60c6a"  width="400">

<br/>

- Achieves higher accuracy with much fewer paramters, indicating efficient use of resources in MobileNet architecture.










## **MobileNet V2**

<br/>

### **ReLU Causes Manifold Collapse**

<br/>

- While ReLU is effective in promoting non-linearity and alleviating the vanishing gradient problem, it can cause the loss of information encoded in negative values as it sets the activations as zero except positive values.

- This becomes signified when the relu is applied on the data compressed in dimensionality.

- **Manifold in Low-Dimensional Subspace of the Activation Space**

    - Assuming that manifold in neural networks is embedded in low-dimensional subspaces, ReLU activation can maintain that input manifold as the type of the activation it performs is limited to linear transformation (mulitply 1 if positive, 0 if negative).
    - But the preservation of input manifold can only be possible if the dimensionality of activation space is much higher than that of the manifold.Otherwise, entire deletion of information embedded in negative values using ReLU results in significant distortion of the manifold (Collapsed manifold)

&emsp;&emsp;&emsp;**Figure 1. ReLU transformation of low-dimensional manifolds into n-dimensional output**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/7320b284-ffe7-4bdf-b218-1ec5727a8274" width="600">

<br/>

- Input : initial input where low-dimensional manifold is embedded
- output dimension 'n' increases from 2 to 30
- **Operations**
    1. embed into n-dimensional space with random matrix $\large T$ (2xn)
    2. follwed by ReLU transformation
    3. restore the data into its original dimensionality with $\large T^{-1}$
    4. plot the data to see if ReLU transformation preserve the manifold or not.

- Can observe expansion into higher dimensionality before ReLU being applied shows better maintenance of input manifold. This mean that input manifold embedded into much lower-dimensional subspace of activation space can be preserved after ReLU transformation.

<br/>

### **Inverted Residual Block**

<br/>

- To tackle this issue, authors developed a unique block called **"Inverted Residual Bottleneck"** where the residual learning undergoes an **expansion layer** before entering into ReLU activation and then spatially filtered with a **lightweight depthwise 3x3 convolution** follwed by **1x1 pointwise convolution with linear activation (no ReLU)**. Skip connection then is added to final resultant output of pointwise convolution and **no ReLU activation applied after concatenation**. This is the exact opposite of how typical bottleneck structure as used in MobileNet V1 operates.

<br/>

&emsp;&emsp;&emsp;**Figure 3. Comparison between original residual block in V1 and inverted residual block in V2**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/5b73b94a-164d-4fbb-af3a-ad58d53fc93b" width="640">

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/a55aa2c4-3c3f-4598-a8fb-847389fe0440" width="500">

<br/>

- This enables implementing non-linearity with ReLU without losing input manifold informaiton.

- Linear activation adopted here plays a crucial as it prevents non-linearities from destroying too much information, resulting in better model performance.

&emsp;&emsp;&emsp;**Figure 6. The impact of non-linearities and various
types of shortcut connections**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/775c754f-3d15-41d1-9c8c-a9d2c516f2aa" width="550">

<br/>

&emsp;&emsp;&emsp;**Table 1. Architecture of Inveted Bottleneck Residual block used in MobileNet V2**

&emsp;&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/ea78ef5b-cd29-481d-bce9-38d9537487a3" width="480"> <br/>

- can see MobileNet V2 has clear computaitonal advantages over other networks with the lowest max number of memory.

- **ReLU6 = min(max(x, 0), 6)**
    - clamp the maximum value as 6
    - increases non-linearity
    - computational stability and efficient use of memory resources

- **Residual connection** only used for the block whose input shape equlas to output shape.


<br/>

### **Computational Advantage of MobileNet V2 over V1**

<br/>

- Computational cost of **depthwise (DW) vs pointwise (PW)** layer
    - using 3x3 convolution to input of 64 channel with 128 output feature maps
    - DW : 3 x 3 x 64
    - PW : 1 x 1 x 64 x 128
    - Typically, PW tends to require greater computations compared to DW
    - New inverted residuals block takes advantage of this by expanding DW layer and narrowing down the PW layer.


- **Comparison : V1 vs V2**
- **Example** <br/>

&emsp;&emsp;&emsp; <img src="https://github.com/SuminizZ/Physics/assets/92680829/899da86f-78ee-4528-a121-19feb0b793a3" width="600">

- Inverted Residual Block (V2) : (1 x 1 x 24 x 144) + (3 x 3 x 144) + (1 x 1 x 144 x 24) = 8,208
- Residuals Block (V1) : (3 x 3 x 144) + (144 x 144) = 22,032

- Inverted residuals block of MobileNet V2 has rougly about 1/3 computations required for MobileNet V1

&emsp;&emsp;&emsp;**Table 3. The max number of channels/memory (in Kb) for different network architectures**

&emsp;&emsp;&emsp;&emsp;&emsp; <img src="https://github.com/SuminizZ/Physics/assets/92680829/08343946-583a-43e3-91f6-803a2534aed9" width="450">

&emsp;&emsp;&emsp;**Table 4. Performance on ImageNet for different architectures**

&emsp;&emsp;&emsp;&emsp;&emsp; <img src="https://github.com/SuminizZ/Physics/assets/92680829/08343946-583a-43e3-91f6-803a2534aed9" width="450">

<br/>

### **MobileNet V2 Architecture**

<br/>

&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/bb6a7863-d3d5-4273-b4f1-4311d67f1d3f" width="440">



## **MobileNet V3**

<br/>

### **1. Adding Squeeze and Excitation Layer**

- Squeeze-Excitation (SE) module
    - can learn interdependent importance between features (channels)

    - Squeeze : GAP into 1x1xC (channels)

    - Excitation : 2 steps FC layer to get stronger representation for important feature or pattern

&emsp;&emsp;&emsp;**Squeeze-Excitation**

&emsp;&emsp;&emsp;&emsp;&emsp; <img src="https://github.com/SuminizZ/Physics/assets/92680829/f19d8cfc-7380-4728-bfab-5ff4c9261c05" width="380">

&emsp;&emsp;&emsp;**MobileNet V2 + SE module**

&emsp;&emsp;&emsp;&emsp;&emsp; <img src="https://github.com/SuminizZ/Physics/assets/92680829/2fa3a0af-da40-4bac-bf92-74fe7c7f9fc1" width="580">

 <br/>

- being interposed between 3x3 depthwise convolution and 1x1 pointwise convolution

<br/>

### **2. Use of New Non-Linearity : Hard Sigmoind & Hard Swish**

- Sigmoid = $\large \frac{1}{1\,+\,e^{-x}}$

- Swish = $\large x\times\,sigmoid(x)$

- Hard Sigmoid = $\large \frac{ReLU6\,(x\,+\,3)}{6}$

- Hard Swish = $\large x\times\, \frac{ReLU6\,(x\,+\,3)}{6}$

&emsp;&emsp;**Figure 6**

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/bf8e1578-7937-42d6-8505-5154114162af" width="580">

- compuationally stable with no possible precision error caused by different implementations of approximate sigmoid function

- ReLU6 as piece-wise function unlike sigmoid significanlty reduces memory use with no discernible difference in accuracy

<br/>

### **3. Redesigning Expensive Layers**

<br/>

&emsp;&emsp;**Figure 5. : Comparision of original last stage and efficient last stage**

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/7548afe8-0f8b-4a9c-b045-622233588b20" width="580">

<br/>

- **Last feature extraction layer**
    - original : **1x1 conv - BN - H-swish** stage that expands the feature from 320 to 1280 to 7x7 resoultion input
        - computations : 7 x 7 x 320 x 1280
        - greatly increases the computations and thus, latency

    - modified : move the feature extraction stage past the final average pooling layer
        - applies **1x1 conv - H-swish** to 1x1x960 input (instead of 7x7 spatial resolution) and creates output with 1280 features
        - computations : 1 x 1 x 960 x 1280

    - The efficient last stage reduces the latency by 7 milliseconds which is 11% of the running time and reduces the number of operations by 30 millions MAdds with almost no loss of accuracy

- **Replace ReLU6 -> H-Swish non-linearity**

    - Use dfferent type of non-linearity to reduce redundancy

    - As a result, able to reduce the number of filters to 16 while maintaining the same accuracy as 32 filters using either ReLU or swish. This saves an additional 2 milliseconds and 10 million MAdds.

<br/>

### **MobileNet V3 Architecture (Small & Large)**

<br/>

- Found by **1. platform-aware NAS for block-wise search** and **2. NetAdapt for layer-wise search**
    - find the one that maximizes the ratio of accuracy change to latency change (maximizes the trade-off slope : $Δ$ accuracy /  $Δ$ latency)

    - proposal type
        - size of expansion layer
        - reduce the bottleneck in all blocks using same bottleneck size to keep residual-connections

<br/>

&emsp;&emsp;**Table 1. :  Specification for MobileNetV3-Large**

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/212f2343-c8bd-4e60-ad8e-e558fb9fc19a" width="500">

<br/>

&emsp;&emsp;**Table 2. :  Specification for MobileNetV3-Small**

&emsp;&emsp;<img src="https://github.com/SuminizZ/Physics/assets/92680829/212f2343-c8bd-4e60-ad8e-e558fb9fc19a" width="500">

<br/>

- All channels are divisible by 8 for computational efficiency


---

## **Implementation with PyTorch**

In [ ]:
def get_divisible(v, divider):
    new_v = (int(v + divider/2)//divider)*divider
    if new_v < 0.9*v:
        new_v += divider
    return max(new_v, divider)


class Squeeze_Excite(nn.Module):
    def __init__(self, channels, r=4):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d((1,1))
        self.excitation = nn.Sequential(nn.Linear(channels, channels//r),
                                        nn.ReLU(inplace=True),
                                        nn.Linear(channels//r, channels),
                                        nn.Hardsigmoid(inplace=True))

    def forward(self, x):
        se = self.squeeze(x)
        se = se.reshape(se.shape[0], -1)
        se = self.excitation(se)
        x *= se.unsqueeze(2).unsqueeze(3)
        return x


class InvResBlock(nn.Module):
    def __init__(self, in_channels, k, exp_channels, out_channels, se, nl, s):
        super().__init__()
        self.se = se
        self.skip_connect = True if s == 1 and in_channels == out_channels else False
        non_linearity = nn.ReLU(inplace=True) if nl=='relu' else nn.Hardswish(inplace=True)

        layers = []
        if in_channels != exp_channels:
            layers += [nn.Sequential(nn.Conv2d(in_channels, exp_channels, 1, bias=False),
                                     nn.BatchNorm2d(exp_channels, momentum=0.99),
                                     non_linearity)]

        layers += [nn.Sequential(nn.Conv2d(exp_channels, exp_channels, k, stride=s, padding=(k-1)//2, groups=exp_channels, bias=False),
                                 nn.BatchNorm2d(exp_channels, momentum=0.99),
                                 non_linearity)]
        if self.se:
            layers += [Squeeze_Excite(exp_channels)]
        layers += [nn.Sequential(nn.Conv2d(exp_channels, out_channels, 1, bias=False),
                                 nn.BatchNorm2d(out_channels, momentum=0.99))]

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        residual_x = self.layers(x)
        if self.skip_connect:
            return x + residual_x
        return residual_x


class MobileNetV3(nn.Module):
    def __init__(self, in_channels, cfgs, num_classes, zero_init_residual=True, width_exp=1.):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, 16, 3, padding=1, stride=2, bias=False),
                                    nn.BatchNorm2d(16, momentum=0.99),
                                    nn.Hardswish(inplace=True))
        in_channels = 16
        layers = []
        fc_out = cfgs[-1]
        for k, exp_channels, out_channels, se, nl, s in cfgs[:-2]:
            layers += [InvResBlock(in_channels, k, get_divisible(exp_channels, 8), get_divisible(out_channels*width_exp, 8), se, nl, s)]
            in_channels = get_divisible(out_channels*width_exp, 8)

        self.blocks = nn.Sequential(*layers)

        k, _, out_channels, self.se, nl, s = cfgs[-2]
        non_linearity = nn.ReLU(inplace=True) if nl=='relu' else nn.Hardswish(inplace=True)

        if self.se:
            self.se_block = Squeeze_Excite(out_channels)
        self.last_conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, k, bias=False),
                                       nn.BatchNorm2d(out_channels),
                                       non_linearity)
        self.gap = nn.AdaptiveAvgPool2d((1,1))
        self.classifier = nn.Sequential(nn.Linear(out_channels, fc_out),
                                        nn.Hardswish(inplace=True),
                                        nn.Dropout(p=0.2, inplace=True),
                                        nn.Linear(fc_out, num_classes))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 1e-2)
                nn.init.zeros_(m.bias)


    def forward(self, x):
        x = self.conv1(x)
        x = self.blocks(x)
        x = self.last_conv(x)
        if self.se:
            x = self.se_block(x)
        x = self.gap(x)
        x = torch.flatten(x, start_dim=1)
        out = self.classifier(x)
        return out


class Config():
    def __init__(self):
        pass

    def large(self):
        # kernel, exp, out, se, nl, s
        cfgs = [(3, 16, 16, False, 'relu', 1),
                (3, 64, 24, False, 'relu', 2),
                (3, 72, 24, False, 'relu', 1),
                (5, 72, 40, True, 'relu', 2),
                (5, 120, 40, True, 'relu', 1),
                (5, 120, 40, True, 'relu', 1),
                (3, 240, 80, False, 'hs', 2),
                (3, 200, 80, False, 'hs', 1),
                (3, 184, 80, False, 'hs', 1),
                (3, 184, 80, False, 'hs', 1),
                (3, 480, 112, True, 'hs', 1),
                (3, 672, 112, True, 'hs', 1),
                (5, 672, 160, True, 'hs', 2),
                (5, 960, 160, True, 'hs', 1),
                (5, 960, 160, True, 'hs', 1),
                (1, None, 960, False, 'hs', 1),
                1280]

        return cfgs

    def small(self):
        cfgs = [(3, 16, 16, True, 'relu', 2),
                (3, 72, 24, False, 'relu', 2),
                (3, 88, 24, False, 'relu', 1),
                (5, 96, 40, True, 'hs', 2),
                (5, 240, 40, True, 'hs', 1),
                (5, 240, 40, True, 'hs', 1),
                (5, 120, 48, True, 'hs', 1),
                (5, 144, 48, True, 'hs', 1),
                (5, 288, 96, True, 'hs', 2),
                (5, 576, 96, True, 'hs', 1),
                (5, 576, 96, True, 'hs', 1),
                (1, None, 576, True, 'hs', 1),
                1024]

        return cfgs


## **Model Summary**


In [ ]:
configs = Config()
cfgs_small = configs.small()
cfgs_large = configs.large()

In [ ]:
large_model = MobileNetV3(3, cfgs_large, 1000)
summary(large_model, input_size=(2, 3, 224, 224), device='cpu')

Layer (type:depth-idx)                             Output Shape              Param #
MobileNetV3                                        [2, 1000]                 --
├─Sequential: 1-1                                  [2, 16, 112, 112]         --
│    └─Conv2d: 2-1                                 [2, 16, 112, 112]         432
│    └─BatchNorm2d: 2-2                            [2, 16, 112, 112]         32
│    └─Hardswish: 2-3                              [2, 16, 112, 112]         --
├─Sequential: 1-2                                  [2, 160, 7, 7]            --
│    └─InvResBlock: 2-4                            [2, 16, 112, 112]         --
│    │    └─Sequential: 3-1                        [2, 16, 112, 112]         464
│    └─InvResBlock: 2-5                            [2, 24, 56, 56]           --
│    │    └─Sequential: 3-2                        [2, 24, 56, 56]           3,440
│    └─InvResBlock: 2-6                            [2, 24, 56, 56]           --
│    │    └─Sequential: 3-3   

In [ ]:
small_model = MobileNetV3(3, cfgs_small, 1000)
summary(small_model, input_size=(2, 3, 224, 224), device='cpu')

Layer (type:depth-idx)                             Output Shape              Param #
MobileNetV3                                        [2, 1000]                 --
├─Sequential: 1-1                                  [2, 16, 112, 112]         --
│    └─Conv2d: 2-1                                 [2, 16, 112, 112]         432
│    └─BatchNorm2d: 2-2                            [2, 16, 112, 112]         32
│    └─Hardswish: 2-3                              [2, 16, 112, 112]         --
├─Sequential: 1-2                                  [2, 96, 7, 7]             --
│    └─InvResBlock: 2-4                            [2, 16, 56, 56]           --
│    │    └─Sequential: 3-1                        [2, 16, 56, 56]           612
│    └─InvResBlock: 2-5                            [2, 24, 28, 28]           --
│    │    └─Sequential: 3-2                        [2, 24, 28, 28]           3,864
│    └─InvResBlock: 2-6                            [2, 24, 28, 28]           --
│    │    └─Sequential: 3-3   

In [ ]:
x = torch.randn(2,3,224,224)
out = large_model(x)
print(out.shape)

torch.Size([2, 1000])
